In [3]:
import nest_asyncio
from browser_use import Agent
nest_asyncio.apply()

INFO     [browser_use] BrowserUse logging setup complete with level info
INFO     [root] Anonymized telemetry enabled. See https://github.com/gregpr07/browser-use for more information.


In [1]:
from langchain_google_vertexai.model_garden import ChatAnthropicVertex
from langchain_google_vertexai import ChatVertexAI

In [2]:
project = "ai-call-bot-440714"
location = "us-east5"

# Initialise the Model
model = ChatAnthropicVertex(
    model_name="claude-3-5-sonnet-v2@20241022",
    project=project,
    location=location,
)

In [4]:
agent = Agent(
    task="Go to https://wargaming.com/en/careers/ and extract the list of all paths of the vacancies by href, if needed click next page button",
    llm=model,
)
result = await agent.run()

INFO     [agent] 🚀 Starting task: Go to https://wargaming.com/en/careers/ and extract the list of all paths of the vacancies by href, if needed click next page button
INFO     [agent] 
📍 Step 1
INFO     [agent] 🤷 Eval: Unknown - Starting fresh on a blank page
INFO     [agent] 🧠 Memory: Need to go to Wargaming careers page and extract vacancy hrefs
INFO     [agent] 🎯 Next goal: Navigate to the Wargaming careers page
INFO     [agent] 🛠️  Action 1/1: {"go_to_url":{"url":"https://wargaming.com/en/careers/"}}
ERROR    [agent] ❌ Result failed 1/3 times:
 Error executing action go_to_url: Page.goto: Timeout 30000ms exceeded.
Call log:
  - navigating to "https://wargaming.com/en/careers/", waiting until "load"

INFO     [agent] 
📍 Step 2
INFO     [agent] ⚠ Eval: Failed - The initial URL navigation timed out, but we landed on the correct page anyway
INFO     [agent] 🧠 Memory: We are on the Wargaming careers page and need to click on Latest job opportunities to find the vacancy listings
INFO    

In [15]:
from playwright.async_api import async_playwright
import asyncio

async def extract_vacancy_paths():
    async with async_playwright() as p:
        browser = await p.chromium.launch(
            headless=True,
            args=[
            '--no-sandbox',
            '--disable-blink-features=AutomationControlled',
            '--disable-infobars',
            '--disable-background-timer-throttling',
            '--disable-popup-blocking',
            '--disable-backgrounding-occluded-windows',
            '--disable-renderer-backgrounding',
            '--disable-window-activation',
            '--disable-focus-on-load',
            '--no-first-run',
            '--no-default-browser-check',
            '--no-startup-window',
            '--window-position=0,0',
            '--window-size=1280,1000',
            '--disable-web-security',
            '--disable-site-isolation-trials',
            '--disable-features=IsolateOrigins,site-per-process'
        ])
        page = await browser.new_page()

        vacancy_paths = []


        try:
            await page.goto("https://wargaming.com/en/careers/", timeout=30000)
            await page.wait_for_load_state('networkidle')
        except Exception as e:
            print(f"Navigation timeout occurred: {e}")
            # Continue execution as the page might still be usable



        # Wait for and click the jobs list link
        try:
            await page.wait_for_selector('a[href="#jobs_list"]', timeout=30000)
            await page.click('a[href="#jobs_list"]')
            await page.wait_for_load_state('networkidle')
        except Exception as e:
            print(f"Selector 'a[href='#jobs_list']' not found: {e}")
            # Continue execution as the page might still be usable

        # Wait for job listings to appear
        try:
            await page.wait_for_selector('.job-listing', timeout=30000)
        except Exception as e:
            print(f"Selector '.job-listing' not found: {e}")
            # Continue execution as the page might still be usable

        # Try different selectors for job links
        selectors = [
            '.job-listing a',
            'a[href*="/careers/vacancy/"]',
            '[data-vacancy-link]',
        ]

        for selector in selectors:
            try:
                links = await page.eval_on_selector_all(
                    selector,
                    "elements => elements.map(el => el.getAttribute('href'))"
                )
                if links:
                    vacancy_paths.extend(links)
                    break
            except Exception as e:
                print(f"Selector {selector} failed: {e}")
                continue
        # Filter out None values and duplicates
        return list(set([path for path in vacancy_paths if path]))

In [16]:
paths = await extract_vacancy_paths()
print(f"Found {len(paths)} unique vacancy paths:")
for path in paths:
    print(path)

Navigation timeout occurred: Page.goto: Timeout 30000ms exceeded.
Call log:
  - navigating to "https://wargaming.com/en/careers/", waiting until "load"

Selector 'a[href='#jobs_list']' not found: Timeout 30000ms exceeded.
=========================== logs ===========================
"load" event fired
Selector '.job-listing' not found: Page.wait_for_selector: Timeout 30000ms exceeded.
Call log:
  - waiting for locator(".job-listing") to be visible

Found 0 unique vacancy paths:
